# Notebook 4.2: Prompt Chaining

Building on the simple agent introduced in [Level 1](4.1_tool_calling.ipynb), this tutorial continues the agent-focused section of our series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to complete multi-step tasks, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. 
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. 

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./3_Llama_Stack.ipynb) notebook. 

In [ ]:
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

from src.client_tools import get_pod_log
from src.lab import get_agent, run_session

Next, we will initialize our environment as described in detail in our ["Llama Stack" notebook](./3_Llama_Stack.ipynb). Please refer to it for additional explanations.

## Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines the use of two tools: location detection and web search.

1. **Automatic Location Detection**: Use the `get_location` client tool (have a look in the src folder `client_tools.py`) to automatically determine the user's current location.
2. **Contextual Search**: Leverage the detected location to formulate the correct websearch query.

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location using `get_location`.
- Then use that location to search for nearby weather-related risks.
- Finally, present a comprehensive response.

This demonstrates how the builtin websearch tool and custom client tools can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [ ]:
instructions = """
    You are a helpful assistant.
    When a user asks about logs from a pod, you MUST use the get_pod_log tool."""
#    When a user asks for general information, you MUST use the websearch tool.
#"""

agent = get_agent(instructions, (get_pod_log, 'builtin::websearch'))

In [ ]:
pod_name = 'java-app-build-iyyx5z-build-pod'
namespace = 'demo-application'
container_name = 'step-s2i-build'

user_prompts = [
    f"Retrieve the logs of the {container_name} container within the {pod_name} pod in namespace {namespace}.",
    "Provide a short summary of these logs and troubleshooting recommendations in bullet points."
]
run_session(agent, 'prompt-chaining-session', user_prompts)